In [ ]:
%reload_ext autoreload
%autoreload 2

In this chapter we will **only talk about classification**. Classifiers associate a transformed set of features to a set of classes (a discrete variable)

![class_reg](../images/class_reg.png)

In this notebook we are going to focus on distance based classification method (opposed to decision tree for example wich works with threshold more than actual distances). The goal is to give you some insight on 4 different levels:
- **Best practices** 
- **Intuition on the concepts behind those methods**
- **How to implement them with Scikitlearn**
- **Intuition on the different parameters that your model need but are not trainable (hyperparameters)**

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

from sklearn.neighbors import KNeighborsClassifier
from IPython.display import Image
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc, precision_recall_curve
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier


from sklearn.multiclass import OneVsOneClassifier
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn import svm


%matplotlib inline

import matplotlib.pylab as pylab
pylab.rcParams['figure.figsize'] = 8, 8
plt.rc("font", size=14)


plt.rc('xtick', color='k', labelsize='medium', direction='in')
plt.rc('xtick.major', size=8, pad=12)
plt.rc('xtick.minor', size=8, pad=12)

plt.rc('ytick', color='k', labelsize='medium', direction='in')
plt.rc('ytick.major', size=8, pad=12)
plt.rc('ytick.minor', size=8, pad=12)

# Distance based classification method
## Table Of Content: <a id='toc'></a>

* [Meet the data](#data)
    * [Binary class dataset : breast cancer](#data-cancer)

* [**K-nearest neighbors : introduction to basic routine in ML**](#neighbors)
    * [Toy dataset: KNN concepts and hyperparameters](#KN-concepts)
    * [a metric to score classification : accuracy](#accuracy)
    * [finding the best hyper-parameter set with KNN](#knn-hyper)
    * [important considerations: leakage](#metric-leakage)
    * [applying all this to the cancer dataset](#knn-cancer)
    
* [**Logistic regression**](#Logistic-regression)
    * [Concepts: what is linear regression and introduction to regularization](#Logistic-regression)
    * [Breast cancer dataset](#LR-hyper)
    * [imbalanced dataset](#imbalanced)    
    
* [**Multiclass problems**](#LR-IRIS)
    
    
* [**Exercise: criticizing code**](#exercise-student)
* [**Exercise: predicting 10 year coronary heart disease outcome**](#exercise-heart)
    

* [**Appendices**](#APPENDIX)
    * [**Support Vector Machine**](#SVM)
    * [**Multiclass dataset : penguins identification**](#data-penguin)
    * [**Exercise**](#exo)




# Meet the data <a id='data'></a>



[Back to the ToC](#toc)

## Binary class dataset : breast cancer <a id="data-cancer"></a>

In the cancer dataset you have 569 tumors for which many features have been measured. The goal is to predict if the tumor is malignant or not.

In [ ]:
# the breast cancer dataset is integrated in the sklearn library
from sklearn.datasets import load_breast_cancer

data = load_breast_cancer()

## we reduce the features because otherwise this problem is a bit too easy ;-)
m = list( map( lambda x : x.startswith("mean ") , data["feature_names"] ) )


X_cancer=data['data'][:,m]

# for some reason this dataset has encoded 0 for malignant,
# which is extremely counter intuitive
# So I choose to switch it to the more intuitive order now
y_cancer= 1-data['target']

#making it into a dataframe
df_cancer=pd.DataFrame(X_cancer,
    columns=data["feature_names"][m])

df_cancer["malignant"]=y_cancer

# 0 : benign
# 1 : malignant


df_cancer.head()

In [ ]:
df_cancer.malignant

In [ ]:
print('features:\n')
for s in df_cancer.columns:
    print(s)

In [ ]:
df_cancer.malignant.value_counts()

So, there is a slight over-representation of benign tumors.

In [ ]:
df_cancer_corr = df_cancer[list(df_cancer.columns)[:-1]].corr()

sns.clustermap(df_cancer_corr,figsize=(10,10),z_score=None,row_cluster=True,col_cluster=True,method='ward',cmap='coolwarm',vmax=1,vmin=-1, annot=True, annot_kws={"size": 13},cbar_kws={"label": 'Pearson\ncorrelation'})

plt.show()

[Back to ToC](#toc)

# K-nearest neighbors +  some basic routine <a class="anchor" id="neighbors"></a>

K-nearest neighbors is a pretty simple algorithm in terms of concept but it already has few hyperparameters that you should  understand and try to optimize. It will introduce you to some of the very experimental like routine that machine learning is.

For a classifier, k-nearest neighbors works as follow. 

**First the algorithm simply saves the labels that it is given during the training phase**. 

Then during the testing phase it takes a testing point and checks its `n_neighbors` nearest neighbors. 

**If `n_neighbors` nearest neighbors are mostly (in majority) from one label then the tested point will be assigned this label**. 

The way the `n_neighbors` **nearest neighbors vote** can be either 

 * uniform (every point as the same importance in the vote) or,
 * distance-based (a point distant to the tested point by a distance d will have a weight of 1/d in the vote).

![knn](../images/knn.png)

[Image from datacamp tutorials](https://www.datacamp.com/community/tutorials/k-nearest-neighbor-classification-scikit-learn)

[Back to the ToC](#toc)

### Toy dataset: KNN concepts and hyperparameters<a class="anchor" id="KN-concepts"></a>

In [ ]:
#Let's make some toy dataset. Here using the sklearn function making blobs for you!!!

from sklearn.datasets import make_blobs

# 120 points, 3 blobs/clusters with some spread=3
blob_centers = np.array([[-7,2.5],[6,-6],[8,-3]])
blob_stds = [[1,3],[1,3],[1,3]]
X_3, y_3 = make_blobs(n_samples = 120, 
                      centers = blob_centers,
                      cluster_std = blob_stds, random_state = 42)


#Random_state is here just to be sure that every time you will get the same blobs. 
# If you change the random_state or do not
# specify it then you will get a new plot every time you call the function (random seed)

In [ ]:
plt.scatter(X_3[:,0],X_3[:,1],c=y_3,cmap=plt.cm.coolwarm,edgecolors='k')
plt.show()

fitting a KNN classifier with sklearn looks like this:

In [ ]:
## create classifier
knn = KNeighborsClassifier( n_neighbors= 5 ) ## how many neighbors to consider

## fitting classifier
knn.fit( X_3 , y_3 )

## getting the classes predicted by the classifier:
knn.predict( X_3 )

In [ ]:
## comparing the prediction and the truth:

pd.crosstab( y_3 , knn.predict( X_3 ) , rownames=['truth'] , colnames=['prediction'] )

Let's look at the effect of the number of neighbors:

In [ ]:
from utils import contour_knn # in utils.py we prepared a number of convenience functions 
fig,axes = plt.subplots(1,3,figsize=(15,5))

## 12 neighbors
contour_knn(120,X_3,y_3,'uniform', ax=axes[0])

## 20 neighbors
contour_knn(20,X_3,y_3,'uniform', ax=axes[1])

## 1 neighbors
contour_knn(1,X_3,y_3,'uniform', ax=axes[2])

In the above we varied the number of voters (neighbors) to decide weither a point is from one class or another. 

You can see how the **boundaries are way more wiggly and attentive to missclassification when the number of neighbors is low**. 

But you can also imagine that if **new data is added it is likely to be missclassified** with those kind of too specific boundaries. 

Conversely, when the number of neighbors is very high (120 here), then the **model is not sensitive to point placement**.

This is a first example of the **bias variance trade off.**

We say that:

 - the model with k=1 has a **high variance**
 - the model with k=120 has a **high bias**


---

Another parameter is **how voting is done**.

Above each of the K nearest neighbors contributed equally to the vote.

By setting `weights='distance'` the contribution of each neighbor will be weighted by the inverse of its distance to the point.

In [ ]:
fig,axes = plt.subplots(1,3,figsize=(15,5))

contour_knn(120,X_3,y_3,'distance', ax=axes[0])
contour_knn(20,X_3,y_3,'distance', ax=axes[1])
contour_knn(1,X_3,y_3,'distance', ax=axes[2])

Do you see what changed?

So here, connecting this to the previous chapter: K and the weighting, which govern our model bias-variance tradeoff and cannot be set directly from the data are **hyper-parameters** of the KNN algorithm.


There are many additional hyper-parameters we could consider as well:

 * the imputation strategy
 * the normalization strategy
 * dimensionality reduction : how many PCA axes do I keep?
 * choice of the classification algorithm (KNN, logistic regression, ranfom forest, neural network, ...)
 
 ...
 
 


[Back to the ToC](#toc)


### A metric to score classification : accuracy <a id="accuracy" ></a>

One straight forward metric to evaluate our model is the accuracy. Accuracy is only interesting on the test set, even though it can give you some good insight about your model when accuracy is compared between test and training set. 

But again, the main thing that is going to matter to evaluate your model concern metric evaluated on the test set.


Accuracy is defined as follow : $\frac{TP+TN}{P+N}$

* TP : True Positive
* TN : True Negative
* FP : False Positive
* FN : False Negative
* P : Positive : $P=TP+FN$
* N : Negative : $N=TN+FP$


In [ ]:
from sklearn.metrics import accuracy_score

truth = [0,0,0,1,1,1]
pred  = [0,0,1,0,1,1] # 2 errors out of 6 predictions

accuracy_score( truth , pred )


[Back to the ToC](#toc)


### finding the best hyper-parameter set with KNN <a id="knn-hyper" ></a>






We implement here the procedure we have seen with the potato dataset in the previous chapter :

**But** we will have to adapt a few things.


First during the train/test split, we use the `stratify` argument to ensure that we have the same proportion of each class in the train and the test set.

In [ ]:
from sklearn.model_selection import train_test_split

X_train , X_test , y_train, y_test = train_test_split(X_3,y_3, test_size=0.25 , 
                                                      stratify=y_3 ) ## stratify  

print('train set size:',len(y_train))
print(' test set size:',len(y_test))

In [ ]:
pd.Series( y_train ).value_counts()

In [ ]:
pd.Series( y_test ).value_counts()

Then we proceed with the hyper-parameter optimization.

**But** (there's always a but), we need to change the metric we are optimizing.

Before we were optimizing for $R^2$, which appropriate for regression tasks, but now we will switch it for **accuracy**.


**micro-exercise:** Change the code below so the gridSearchCV :
 * operates on accuracy instead of $R^2$. *Hint:* you can look up the [documentation of GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html).
 * explores hyper-parameters of the KNN *Hint: remember the double _ between step name and parameter name *:
     * `weights` : can take values uniform or distance
     * `n_neighbors` : can take values between 1 and 50


In [ ]:
%%time
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

pip_knn = Pipeline([('scaler',StandardScaler()),
                    ('model',KNeighborsClassifier())])


grid_values = { ... } ## set up the hyper-parameters here


grid_knn = GridSearchCV(pip_knn, 
                        param_grid = grid_values, 
                        scoring= ... , ## add the proper score here
                        cv = 5,
                        n_jobs=-1)


grid_knn.fit(X_train, y_train)


print(f'Grid best score ({grid_knn.scoring}): {grid_knn.best_score_:.3f}')


print('Grid best parameter :')
for k,v in grid_knn.best_params_.items():
    print(' {:>20} : {}'.format(k,v))


<br>
---
<br>

Once we have a final model, we can evalutae how it performs on the test set

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix
## gridSearch CV fits a new estimator with the best hyperparameter values
reg = grid_reg.best_estimator_

y_pred = reg.predict( X_train )
print(f"train data accuracy: { accuracy_score( y_train , y_pred ) :.2f}")

y_test_pred = reg.predict( X_test )

print(f" test data accuracy: { accuracy_score( y_test , y_test_pred ) :.2f}")


## we can create a confusion matrix:
confusion_m = confusion_matrix(y_test, y_test_pred)

plt.figure(figsize=(5.5,4))
sns.heatmap(confusion_m, annot=True)
plt.title('test {} : {:.3f}'.format( grid_knn.scoring , 
                                    accuracy_score( y_test , y_test_pred ) ))
plt.ylabel('True label')
plt.xlabel('Predicted label')


--- 

It seems that here we get a *perfect* performance on the test set.

Why perfect and not slightly below 0 ? 

This is likely linked to the test set very small size (only 10 points per category) which gives us a very **coarse grained estimate of test accuracy**.
 
> You can read more on this subject in this [blog post](https://www.r-bloggers.com/2021/01/what-is-a-good-test-set-size/). 



[Back to the ToC](#toc) 

### important considerations: leakage <a class="anchor" id="metric-leakage"></a>

The **test set should never be touched until the last step which is the model evaluation**. 

By doing so we can be confident that our evaluation of the **ability of our model to generalize to new data** is as fair as it can be.

We say that no information coming from the test set should leak into the train set. If this is the case, we are biasing our understanding of the generalizability of our model. 

To avoid **leakage** you should ensure your test set is absent from even the early stages of your pipeline, such as imputation or feature selection (so you have guessed it, most of the operations we have done until now have lead to leakage...).

[Back to the ToC](#toc)

## applying all this to the cancer dataset <a class="anchor" id="knn-cancer"></a>

**We first divide our data into a train and a test set:**


In [ ]:
from sklearn.model_selection import train_test_split

#default if 75% training, 25% testing
X_cancer_train, X_cancer_test, y_cancer_train, y_cancer_test = train_test_split(X_cancer, 
                                                                                y_cancer,
                                                                                random_state=7,
                                                                                stratify=y_cancer) 

print("number of samples:")
print( f"\tfull  dataset: {len(y_cancer)}" )
print( f"\ttrain dataset: {len(y_cancer_train)}" )
print( f"\ttest  dataset: {len(y_cancer_test)}" )


# train_test_split stratify make sure to split the data such 
# that the two partitions have similar proportion of each target classes
print("\nproportion of malign cancers:")
print( f"\tfull  dataset: {y_cancer.mean():.3f}" )
print( f"\ttrain dataset: {y_cancer_train.mean():.3f}" )
print( f"\ttest  dataset: {y_cancer_test.mean():.3f}" )



In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

# create a basic pipeline : just a KNN classification
pipeline_knn=Pipeline([('model',KNeighborsClassifier(n_jobs=-1))])



# define the hyperparameters you want to test
# with the range over which you want it to be tested. 
grid_values = {'model__n_neighbors': np.arange(1,100,1),
               'model__weights':['uniform','distance']}
# the double underscore (__) is used to refer to a step of the pipeline, here the model.



#Feed it to the GridSearchCV with the right
#score over which the decision should be made

grid_knn_acc = GridSearchCV(pipeline_knn, 
                            param_grid = grid_values, 
                            scoring='accuracy',
                            cv= 10 ,    # 10-fold cross validation
                            n_jobs=-1)  # use all available CPUs
 


## now we fit the models 
grid_knn_acc.fit(X_cancer_train, y_cancer_train)

#get the best parameters
print('Grid best parameter (max. accuracy):\n\t ', grid_knn_acc.best_params_)
#get the best score calculated from the training/validation dataset
print('Grid best score (cross-validated accuracy): {:.3f}'.format( grid_knn_acc.best_score_ ) )

So there it is: we have applied cross-validation to explore the space of hyper-parameters and we have gotten hyper-parameter values that provide a good bias-variance trade-off.

---

But, before we go to the next part, there is one important thing we forgot in this pipeline: 
like PCA or KMeans, the **KNN algorithm relies on distances** between points.

Meaningful distances can only be computed if all features follow a comparable scale, which is not the case in our cancer dataset:

In [ ]:
df_cancer.head()

So, we will need to add a scaling step to our `Pipeline`.

In [ ]:
%%time
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

pipeline_knn=Pipeline([('scale',StandardScaler()), # scaling 
                       ('model',KNeighborsClassifier(n_jobs=-1))]) # KNN


## no new parameters to add for the scaling
grid_values = {'model__n_neighbors': np.arange(1,100,1),
               'model__weights':['uniform','distance']}

grid_knn_acc = GridSearchCV(pipeline_knn, 
                            param_grid = grid_values, 
                            scoring='accuracy',
                            cv= 10 ,            # 10 fold CV
                            n_jobs=-1)          # use all CPUs

## fit the models 
grid_knn_acc.fit(X_cancer_train, y_cancer_train)


print('Grid best parameter (max. accuracy):\n\t ', grid_knn_acc.best_params_)
print('Grid best score (cross-validated accuracy): {:.3f}'.format( grid_knn_acc.best_score_ ) )

Not only does the model make more sense, but the cross-validated accuracy is better now.


Note that the **proper practice is to include the pre-processing step in the Pipeline**. This:
- prevents you from forgetting to apply it in practice
- let you **explore hyper-parameters** of your preprocessing with the gridSearchCV (eg, nume of PCA axes to keep)
- ensure there is **no leakage from the validation set** during the Cross-Validation



When the number of hyper-parameters is low, we can plot how they influence the score. 

This is also useful to see if there are alternative to our best fit:

In [ ]:
df_res = pd.DataFrame( grid_knn_acc.cv_results_ )
#df_res.loc[:, [ 'param_model__n_neighbors', 'param_model__weights',  'mean_test_score', 'std_test_score' ]]


fig,ax = plt.subplots(1,1,figsize=(10,7))
for W in ['uniform','distance']:
    m = df_res.param_model__weights==W
    ax.plot( df_res.loc[m , 'param_model__n_neighbors'] , df_res.loc[m , 'mean_test_score'] , 
               label = W )
    ax.fill_between( np.array( df_res.loc[m , 'param_model__n_neighbors'] , dtype = int ), 
                       np.array( df_res.loc[m , 'mean_test_score']  - df_res.loc[m , 'std_test_score'] ),
                       np.array( df_res.loc[m , 'mean_test_score']  + df_res.loc[m , 'std_test_score']) , 
                    alpha = 0.2 )
ax.set_xlabel('number of neighbors')
ax.set_ylabel('accuracy')           
ax.legend()    

We see here that a handful of hyperparameter sets have displayed the same cross-validated performance.

The common practice in this case is to use the least complex of the sets, which here corresponds to what the gridSearchCV object proposes.

---

Once we have found the best set of hyper-parameters using cross-validation, we can evalutae our final model on the test set.

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix

y_test_score=grid_knn_acc.score(X_cancer_test,y_cancer_test)

print('Grid best parameter (max. accuracy) model on test: ', y_test_score)

y_cancer_pred_test = grid_knn_acc.predict(X_cancer_test)

confusion_m_cancer = confusion_matrix(y_cancer_test, y_cancer_pred_test)

plt.figure(figsize=(5.5,4))
sns.heatmap(confusion_m_cancer, annot=True)
plt.title('test {} : {:.3f}'.format( grid_knn_acc.scoring , y_test_score ))
plt.ylabel('True label')
plt.xlabel('Predicted label')

**Congrats!!!! You just trained a classical machine learning model using best practice in term of scaling, hyperparameter choice, and data leakage.**

You just followed the routine that will be ours in this whole course.

In [ ]:
## you can access the best model of the grid search with :
grid_knn_acc.best_estimator_

Now you might want to save your beautiful model so you can use it later on:

In [ ]:
model = grid_knn_acc.best_estimator_

import pickle
pickle.dump(model, open("my_super_model.pickle", 'wb'))

In [ ]:
## if you want to load it again

loaded_model = pickle.load(open("my_super_model.pickle", 'rb'))
loaded_model

### A few important points

 * the GridSearch algorithm is a nice algorithm but it is [not the only option for hyper-parameter space exploration](https://scikit-learn.org/stable/modules/grid_search.html)
 * K-fold Cross-Validation is ihnerently stochastic, it is common to [repeat it many times](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RepeatedStratifiedKFold.html#sklearn.model_selection.RepeatedStratifiedKFold)
 * Accuracy is rarely the most appropriate metric (but more on that later...)


[Back to ToC](#toc)

# Logistic regression <a class="anchor" id="Logistic-regression"></a>

**The main hypothesis of logistic regression is that the log odds ratio of the probability that a point is part of a certain class follows a linear combination of the features describing this point.** This is equivalent to the assumption that the two classes are linearly separable and that the log odds ratio increases linearly with the distance from the separating line.

Let's say you only have two classes and so your target variable can only take two values $y_{i} \in\{1,0\}$. Let's define $p(\pmb{x})=P(y=1)$ the probability than your datapoint $\pmb{x}$ belongs to class 1. Let's also say that you have $M$ features to describe a point $\{\pmb{x}_{i}\}_{i=1,...,M}$. And that you have N points.

Then we make the hypothesis that:

$log{\frac{p}{1-p}}=w_{0}+\Sigma^{M}_{i=1}w_{i}x_{i}=w_{0}+\pmb{x}\cdot\pmb{w}$

which translates to

$p(\pmb{x}|y=1)=\frac{1}{1+e^{-(w_{0}+\pmb{x}\cdot\pmb{w})}}$  


![logreg](../images/lr.png)

**So the probability that your datapoint is in class 1 is the logistic function (sigmoid) applied to the linear combination of features.** The larger the weights $\pmb{w}$ the steeper the change between the two classes. The values of $\pmb{w}$ are stored by the 'coeff_' attribute of the [sklearn.linear_model.LogisticRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression) class. $w_{0}$ is stored in 'intercept_'. The function 'decision_function($\pmb{x}$)' returns $w_{0}+\pmb{x}\cdot\pmb{w}$ and 'predict_proba($\pmb{x}$)' returns $p(\pmb{x}|y=1)$ and $p(\pmb{x}|y=0)$

Unlike KNN the logistic regression model provides valuable information on the importance of features for the classification. The bigger the absolute value of the weight $w_{i}$ associated with a feature, the more important this feature is to discriminate between your classes (supposed that features are normalized and mapped onto a similar scale).

Fitting a logistic regression model corresponds to optimizing the loss function

$\pmb{w},w_{0}=argmin_{\pmb{w},w_{0}}\mathcal{L}(\pmb{w},w_{0},\pmb{X},\pmb{y})$
<br><br>
$\mathcal{L}(\pmb{w},w_{0},\pmb{X},\pmb{y})=\Sigma^{N}_{i=1}y_{i}\log(p(\pmb{x}_i|y=1))+(1-y_{i})\log(p(\pmb{x}_i|y=0))$

So, now the way we get the w is from fitting our distribution of probability that a point is in class 1. If the number of features is high and there is a significant amount of non-informative features, the fitting procedure can become unstable. What happens is that some weights of non-informative features may become very large in order to minimize the loss function on the training data, which may lead to reduced performance on the test data (overfitting).

In order to avoid this situation we can add a penalty to the loss function that avoids that weights $w_{i}$ become too large. In ML three such penalties are commonly used:

$\mathcal{L}(\pmb{w},w_{0},\pmb{X},\pmb{y}) + \frac{1}{C}\Sigma^{N}_{i=1}|w_{i}|$ , l1 regularization (Lasso) C being the inverse of the weight that you put on that regularization 

$\mathcal{L}(\pmb{w},w_{0},\pmb{X},\pmb{y}) + \frac{1}{C}\Sigma^{N}_{i=1}w_{i}^{2}$ , l2 regularization (Ridge) 

$\mathcal{L}(\pmb{w},w_{0},\pmb{X},\pmb{y}) + \frac{1}{C}\Sigma^{N}_{i=1}(\alpha|w_{i}|+(1-\alpha)w_{i}^{2})$ , elasticnet

For a deeper understanding of those notions :

 - https://www.datacamp.com/community/tutorials/tutorial-ridge-lasso-elastic-net

 - https://towardsdatascience.com/regularization-in-machine-learning-76441ddcf99a




* **L1 (lasso)** regularization produces many zero weights.
    It is useful for the case where you suspect that only a few features are informative. 
* **L2 (ridge)** regularization leads to many features with small, but non-zero weights. 


The L1 regularization may have more bias, and the L2 more variance. 

The elasticnet interpolates between these two situations at the cost of having and additional parameter $\alpha$.



In [ ]:
x = np.array( df_cancer.loc[:,['mean radius']] )
x_norm = StandardScaler().fit_transform(x)
y = df_cancer['malignant']

fig,ax = plt.subplots(1,1, figsize = (14,5) )
ax.scatter( x_norm , y , c = y )
for alpha in [0.01,0.1,1,10]:
    
    # this implementation does not take alpha but rather C = 1/alpha
    C = 1/alpha
    lr = LogisticRegression( penalty = 'l2' , C = C  , solver='liblinear')
    lr.fit(x_norm , y)
    
    proba = lr.predict_proba(np.linspace(x_norm.min()-1,x_norm.max()+1,100).reshape(-1, 1))
    ax.plot( np.linspace(x_norm.min()-1,x_norm.max()+1,100) , proba[:,1] , label = 'alpha = {}'.format(alpha) )
ax.legend()

Before we do the fully fledged  GridSearchCV, 
we do a little demo of the **effect of L1 and L2 regularization on model parameters**:

In [ ]:
## first we split in train and valid
X_train, X_valid, y_train, y_valid = train_test_split(X_cancer_train, 
                                                      y_cancer_train,
                                                      random_state=13245,
                                                      test_size=0.5,
                                                      stratify=y_cancer_train) 

## scaling 
scaler = StandardScaler()
X_train_norm = scaler.fit_transform(X_train)
X_valid_norm = scaler.fit_transform(X_valid)



In [ ]:
feature_names = list( df_cancer.columns) [:-1]

logCs = []

coef_dict = {'name' : [],
             'val' : [],
             'log_C' : []}
accuracies = []

for C in np.logspace(-3,2,100):

    lr = LogisticRegression( penalty = 'l1' , C = C  , solver='liblinear')
    lr.fit(X_train_norm , y_train)
    
    logCs.append(np.log10(C))
    accuracies.append( accuracy_score( y_valid , lr.predict(X_valid_norm) ) )
    
    coef_dict['name'] += list( feature_names )
    coef_dict['val'] += list( lr.coef_[0] )
    coef_dict['log_C'] += [np.log10(C)]* len(feature_names )

coef_df = pd.DataFrame(coef_dict)

In [ ]:
bestC = logCs[ np.argmax( accuracies ) ]

fig,ax = plt.subplots(1,2,figsize = (20,10))

ax[0].plot(logCs , accuracies)
ax[0].set_xlabel("log10( C )")
ax[0].set_ylabel("validation accuracy")
ax[0].axvline( bestC, color='r', ls = '--' )

sns.lineplot( x = 'log_C' , y='val' , hue = 'name' , data= coef_df , ax = ax[1])
ax[1].axvline( bestC , color='r', ls = '--' )

fig.suptitle("logistic regression of cancer data with an L1 regularization.")


**micro-exercise:** We adapted the code above to generate this plot with an **L2** penalty. How do you interpret the difference?

In [ ]:
feature_names = list( df_cancer.columns) [:-1]

logCs = []

coef_dict = {'name' : [],
             'val' : [],
             'log_C' : []}
accuracies = []

for C in np.logspace(-4,1,100):

    lr = LogisticRegression( penalty = 'l2' , C = C  , solver='liblinear')
    lr.fit(X_train_norm , y_train)
    
    logCs.append(np.log10(C))
    accuracies.append( accuracy_score( y_valid , lr.predict(X_valid_norm) ) )
    
    coef_dict['name'] += list( feature_names )
    coef_dict['val'] += list( lr.coef_[0] )
    coef_dict['log_C'] += [np.log10(C)]* len(feature_names )

coef_df = pd.DataFrame(coef_dict)
bestC = logCs[ np.argmax( accuracies ) ]

fig,ax = plt.subplots(1,2,figsize = (20,10))

ax[0].plot(logCs , accuracies)
ax[0].set_xlabel("log10( C )")
ax[0].set_ylabel("validation accuracy")
ax[0].axvline( bestC, color='r', ls = '--' )

sns.lineplot( x = 'log_C' , y='val' , hue = 'name' , data= coef_df , ax = ax[1])
ax[1].axvline( bestC , color='r', ls = '--' )

fig.suptitle("logistic regression of cancer data with an L2 regularization.")


[Back to ToC](#toc)

## logistic regression on the breast cancer dataset  <a class="anchor" id="LR-hyper"></a>

In [ ]:
logi_r=LogisticRegression(solver='liblinear',n_jobs=1) 
## the liblinear solver supports both L1 and L2
# for more details see: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

pipeline_lr=Pipeline([('scalar',StandardScaler()), 
                      ('model',logi_r)])



grid_values = {'model__C': np.logspace(-5,2,200),
               'model__penalty': ['l1','l2'] }
# define the hyperparameters you want to test
# with the range over which you want it to be tested.


# Feed it to the GridSearchCV with the right
# score(here accuracy) over which the decision should be taken
grid_lr_acc = GridSearchCV(pipeline_lr, 
                           param_grid = grid_values, 
                           scoring='accuracy',
                           cv=10, 
                           n_jobs=-1)


## this cell throws a lot of warning, I remove them with the lines under
grid_lr_acc.fit(X_cancer_train, y_cancer_train)

In [ ]:
print('Grid best parameter (max. accuracy): ', grid_lr_acc.best_params_)#get the best parameters
print('Grid best score (cross-validated accuracy): {:.4f}'.format( grid_lr_acc.best_score_ ) )

We could now compare this to the best KNN classifier we obtained

In [ ]:
print('Grid best score (cross-validated accuracy): {:.4f}'.format( grid_knn_acc.best_score_ ) )

It looks like cross-validated performance of both models are very similar here.


Let's check how the logistic regression actually discriminates the two classes with the code below. 

Or said differently : **what can we learn from our model**, apart from pure classification?

In [ ]:
from operator import itemgetter
lr = grid_lr_acc.best_estimator_[1]
w=lr.coef_[0]#get the weights

featureW = pd.DataFrame( {'feature':df_cancer.columns[:-1],'weight':w} )

featureWsorted = featureW.sort_values(by=['weight'] , 
                                      ascending=False , 
                                      key=lambda col : col.abs())

# get the non-null ones
print('Features sorted per importance:')
print( featureWsorted.loc[ featureWsorted["weight"] !=0 ] )


<br>

[Back to the ToC](#toc)

<br>

## Imbalanced dataset <a class="anchor" id="imbalanced"></a> 

Let's use the same small example as before, but now instead of 300 sample of each class, imagine we only have 30 of class 1:

In [ ]:
X1 = np.concatenate( [ np.random.randn(300) , np.random.randn(30)+2 ])
y = np.array( [0]*300 + [1]*30 )

# do not forget to scale the data
X1_norm = StandardScaler().fit_transform(X1.reshape( X1.shape[0] , 1 ))

fig,ax = plt.subplots(1,2, figsize = (14,5) )

sns.histplot( x=X1,hue = y , ax =ax [0])


ax[1].scatter( X1_norm , y , c = y )

for alpha in [0.01,0.1,1,10]:
    
    # this implementation does not take alpham but rather C = 1/alpha
    C = 1/alpha
    lr = LogisticRegression( penalty = 'l2' , C = C )
    lr.fit(X1_norm , y)
    
    proba = lr.predict_proba(np.linspace(-2,3,100).reshape(-1, 1))
    ax[1].plot( np.linspace(-2,3,100) , proba[:,1] , label = 'alpha = {}'.format(alpha) )
ax[1].legend()

You can see that now the point where the probability curves for different alpha converge is not 0.5 anymore...

Also, the probability says fairly low even at the right end of the plot.

In [ ]:
y_predicted = lr.predict(X1_norm)
print( f"Accuracy with a threshold of 0.5 : {accuracy_score(y,y_predicted):.2f}"  )
pd.crosstab( y , y_predicted )

So, most of the class 1 samples are miss-classified (22/30), but we still get a very high accuracy...

That is because, by contruction, both the **logistic regression and accuracy score do not differentiate False Positive and False Negative**.

And the problem gets worse the more imbalance there is :

In [ ]:
from sklearn.metrics import recall_score

## RECALL = TP / (TP + FN)

recall_list = []
acc_list = []

alpha = 1

## trying many values of imbalance
imbalance_list = np.linspace(0,0.99,50)
for imbalance in imbalance_list:

    ## generating the imbalanced data
    n0 = 300
    n1 = int( n0 * (1 - imbalance) )
    if n1 == 0:
        n1 = 1
    
    X1 = np.concatenate( [ np.random.randn(n0) , np.random.randn(n1)+2 ])
    y = np.array( [0]*n0 + [1]*n1 )

    X1_norm = StandardScaler().fit_transform(X1.reshape( X1.shape[0] , 1 ))
    
    
    ## fitting the logistic regression
    lr = LogisticRegression( penalty = 'l2' , C = 1/alpha )
    lr.fit(X1_norm , y)
    
    y_predicted = lr.predict(X1_norm)

    ## computing accuracy and recall
    recall_list.append( recall_score( y , y_predicted ) )
    acc_list.append( accuracy_score(y,y_predicted) )

        
fig,ax=plt.subplots(figsize = (10,4))
ax.plot( imbalance_list , acc_list , label='accuracy' )
ax.plot( imbalance_list , recall_list , label='recall' )
ax.set_xlabel("imbalance")
ax.legend()

So not only does the precision get worse, the **accuracy actually gets higher as there is more imbalance!**

So the problem here may be 2-fold:
 * imbalance in our dataset skews the **logistic regression** toward a particular outcome
 * **accuracy** is not able to differenciate between False Positive and False Negative, and so it is **blind to imbalance**

Consequently, the solutions will have to come both from the model, and from the metric we are using.


**For the logistic regression**:
 * we will re-weight sample according to their class frequency, so that they are more important during the fitting.
 * in sklearn : `LogisticRegression( ... , class_weight='balanced')`
 
<br> 

**For the metric**, there exists several metrics which are sensitive to imbalance problems. 
Here we will introduce some of the main ones:

- **[balanced accuracy](https://scikit-learn.org/stable/modules/model_evaluation.html#balanced-accuracy-score)**:

$$balanced\_accuracy = 0.5*( \frac{TP}{TP+FN} + \frac{TN}{TN+FP} )$$

- **[average-precision score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.average_precision_score.html#sklearn.metrics.average_precision_score)**: weighted mean of precisions achieved at each recall threshold


Precision : $\frac{\bf{TP}}{\bf{TP}+\bf{FP}}$

Recall : $\frac{\bf{TP}}{\bf{TP}+\bf{FN}}$



<img src="../images/precision_recall_curve.png" alt= "precision-recall curve" width="500px">

- **[ROC AUC](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html#sklearn.metrics.roc_auc_score)**: Area Under the Receiver Operating Characteristic Curve 

<img src="../images/ROC_curve.png" alt= "ROC" width="500px">



In [ ]:
from sklearn.metrics import balanced_accuracy_score, recall_score , average_precision_score , roc_auc_score


def check_imbalance_effect( imbalance_list , class_weight = None):
    
    recall_list = []
    balanced_acc_list = []
    acc_list = []
    avg_prec_list = []
    roc_auc_list = []
    
    for imbalance in imbalance_list:

        n0 = 300
        n1 = int( n0 * (1 - imbalance) )
        if n1 == 0:
            n1 = 1

        X1 = np.concatenate( [ np.random.randn(n0) , np.random.randn(n1)+2 ])
        y = np.array( [0]*n0 + [1]*n1 )

        X1_norm = StandardScaler().fit_transform(X1.reshape( X1.shape[0] , 1 ))

        # LR
        lr = LogisticRegression( penalty = 'l2' , C = 1 , class_weight=class_weight)
        lr.fit(X1_norm , y)

        y_predicted = lr.predict(X1_norm)

        recall_list.append( recall_score( y , y_predicted )  )
        acc_list.append( accuracy_score(y,y_predicted) )
        balanced_acc_list.append( balanced_accuracy_score(y,y_predicted) )
        avg_prec_list.append( average_precision_score(y,y_predicted) )
        roc_auc_list.append( roc_auc_score(y,y_predicted) )

    return {'recall':recall_list , 
            'accuracy':acc_list , 
            'balanced_accuracy':balanced_acc_list , 
            'average_precision':avg_prec_list ,
            'ROC_AUC':roc_auc_list}

In [ ]:
imbalance_list = np.linspace(0,0.99,50)

fig,ax=plt.subplots(1,2,figsize = (12,4))

for i,class_weight in enumerate([None,'balanced']):

    scores = check_imbalance_effect( imbalance_list , class_weight = class_weight)

    for scoreName in scores:
        ax[i].plot( imbalance_list , scores[scoreName] , 
                   label=scoreName )
    ax[i].set_xlabel("imbalance")
    ax[i].set_ylim(0,1)
    ax[i].set_title('class_weight={}'.format(class_weight))
    ax[i].legend()


So, aside from accuracy, the metrics are able to detect an imbalance problem.

Setting `class_weight='balanced'` in our logistic regression fixes the imbalance at the level of the model,
which is reflected in several metrics, except the **average-precision**.

Let's look at that:

In [ ]:
from sklearn.metrics import precision_score
from matplotlib import gridspec

fig=plt.figure(figsize = (12,6))
ax0 = plt.subplot2grid((3, 3), (0, 0), colspan=2, rowspan=3)
axes = [ plt.subplot2grid((3, 3), (0, 2)) , plt.subplot2grid((3, 3), (1, 2)) , plt.subplot2grid((3, 3), (2, 2)) ]

for i,imbalance in enumerate( [0.1,0.5,0.8] ) : 

    ## setup 
    n0 , n1 = 300 , max( 1 , int( 300 * (1 - imbalance) ))
    
    X1 = np.concatenate( [ np.random.randn(n0) , np.random.randn(n1)+2 ])
    y = np.array( [0]*n0 + [1]*n1 )
    X1_norm = StandardScaler().fit_transform(X1.reshape( X1.shape[0] , 1 ))

    # LR
    lr = LogisticRegression( penalty = 'l2' , C = 1 , class_weight='balanced')
    lr.fit(X1_norm , y)

    ## precision-recall curve
    y_scores = lr.decision_function(X1_norm)#decision_function gives you the proba for a point to be in
    prec, rec, thre = precision_recall_curve(y, y_scores)
    ax0.plot( rec, prec , label='imbalance={}'.format(imbalance) )    
    
    ## confusion matrix
    y_pred = lr.predict(X1_norm)
    confusion_m = confusion_matrix(y, y_pred)
    sns.heatmap(confusion_m, annot=True , ax = axes[i],fmt='.0f', cmap="crest")

    axes[i].set_ylabel('True label')
    axes[i].set_xlabel('Predicted label')
    
    print('imbalance={}'.format(imbalance) , 
          'recall {:.2f}\tprecision {:.2f}'.format(recall_score(y , y_pred) , precision_score(y , y_pred)) )
ax0.legend()
ax0.set_xlabel('recall')
ax0.set_ylabel('precision')

fig.tight_layout()

So the **average-precision is still sensitive to imbalanced with balanced class weights** because precision **is a ratio across classes: TP and FP**.

In contrasts, the others only use in-labels metrics : TP and FN , TN and FP


In practice, how you handle imbalance should depend on the nature of your data and your problem.

Is the imbalance you see in your data representative of what it will on new data?

Do you care about the type of error you make, or you just want to make as little error as possible ? in the second case, simple accuracy might serve you better

Do you have pre-defined costs associated to different type of errors (eg. [DALY](https://www.who.int/data/gho/indicator-metadata-registry/imr-details/158) of having un-needed surgery vs. DALY of not having needed surgery) ? In that case you may want to use these instead of class frequencies in the balancing and scoring.



## micro-exercise: logistic-regression 

Now that you know about imbalance, adapt the code below to train a logistic regression classifier on the cancer dataset to optimize ROC AUC and properly account for imbalance:

In [ ]:
logi_r=LogisticRegression(solver='liblinear',n_jobs=1) 

pipeline_lr=Pipeline([('scalar',StandardScaler()), 
                      ('model',logi_r)])



grid_values = {'model__C': np.logspace(-5,2,200),
               'model__penalty': ['l1','l2'] }

grid_lr_acc = GridSearchCV(pipeline_lr, 
                           param_grid = grid_values, 
                           scoring='accuracy',
                           cv=10, 
                           n_jobs=-1)


grid_lr_acc.fit(X_cancer_train, y_cancer_train)

In [ ]:
#%load solutions/solution_02_ME3.py

[Back to ToC](#toc)

## Multiclass problems <a class="anchor" id="LR-IRIS"></a>


Many classification problems involve more than 2 categories. 

For example, remember the iris dataset:

In [ ]:
from sklearn.datasets import load_iris
from sklearn.decomposition import PCA
Xi,yi = load_iris(return_X_y = True , as_frame = True)

## simple preprocessing pipeline : scale then PCA
iris_prepro = Pipeline([('scalar',StandardScaler()),('PCA',PCA())])
iris_prepro.fit(Xi,yi)

X_pca = iris_prepro.transform(Xi)
sns.scatterplot( x = X_pca[:,0] , y = X_pca[:,1] , hue = yi.astype(str))


If we try to train a Logistic regression, optimizing for ROC AUC, we get an error:

In [ ]:
pipeline_lr=Pipeline([('scalar',StandardScaler()),
                     ('model',LogisticRegression(class_weight='balanced'))])

#hyper parameter space :
grid_values = {'model__C': np.logspace(-2,1,50),
               'model__penalty':['l1','l2'],
               'model__solver':['liblinear']}

# note the 'roc_auc_ovr' score!
grid_iris = GridSearchCV(pipeline_lr, 
                             param_grid = grid_values, 
                             scoring='roc_auc',n_jobs=-1)

grid_iris.fit(Xi, yi)

print('Grid best parameter (max. roc_auc):')
print( '\t' + '\n\t'.join([str(x) for x in grid_iris.best_params_.items()]))
print('Grid best score (roc_auc): ', grid_iris.best_score_)

LR, like many other models, is, originally, really about classifying 0 and 1, so binary.

Of course all of that can still be applied to a multi-class classification, with a little bit of adaptation.

There are many different ways of tackling the problem, which end up being combinations of these 3 elements :

* Either you treat the problem as **one class vs one class** (ie, you re-encode your data)
* Or you treat the problem as a **one class vs the rest** : you subdivide the problem into as many problem as there are classes
* You change your loss function to a **multinomial one**.

In any case you need to decide how you are going to **agglomerate those different metrics** (different ROC curves for example):

 * **micro average** : pull all raw numbers together (eg. number of FP, TP), group them and then calculate your overall statistic (eg. TPR)
 * **macro average** : calculate each statistics separately and then do the average.

Think about the differences induced by those metrics. Why should you use one more than the other? Or maybe you should always use all of them?

> Spoiler: is has to do with overall separability and balance between the different class.

Previous versions of scikit-learn gave multiple options regarding how to handle multiclass cases, but now they only use the multinomial implementation. 

Regardless, other methods than the logistic regression still have various option, so it pays to be aware of this issue.

In [ ]:
from utils import contour_lr
#multinomial implementation 
contour_lr('l2',X_3,y_3,10)

In [ ]:
## more regularization
contour_lr('l2',X_3,y_3,10**-2)

Let's apply this to the Iris dataset:

In [ ]:

pipeline_lr=Pipeline([('scalar',StandardScaler()),
                     ('model',LogisticRegression(class_weight='balanced'))])

#hyper parameter space :
grid_values = {'model__C': np.logspace(-2,1,50),
               'model__penalty':['l1','l2'],
               'model__solver':['liblinear']}

# note the 'roc_auc_ovr' score!
grid_iris = GridSearchCV(pipeline_lr, 
                             param_grid = grid_values, 
                             scoring='roc_auc_ovr_weighted',n_jobs=-1)

grid_iris.fit(Xi, yi)

print('Grid best parameter (max. roc_auc_ovr_weighted):')
print( '\t' + '\n\t'.join([str(x) for x in grid_iris.best_params_.items()]))
print('Grid best score (roc_auc_ovr_weighted): ', grid_iris.best_score_)


[Back to ToC](#toc)

# Exercise: criticizing code <a class="anchor" id="exercise-student"></a>

Here, rather than code yourself the analysis, we want you to take a look at the following analysis and

1. list the potential methodological issues (you should see at least 3)
2. propose alternative to follow best practices.

Implementing these best practices can be done, as a form of extra question if you have the time.

> The goal is not performance here, we want to focus on the methodological problems. 

---

**The data:**

This [student dataset from kaggle](https://www.kaggle.com/datasets/devansodariya/student-performance-data/data) was obtained in a survey of  395 students' math course in secondary school.

The dataset contains various features about the student socio-economical status and relationship to studies,
like the student age, and gender, their family size, parents occupation, whether they have received paid tutoring, internet access or alcohol consumption...

We are not going to focus on the feature interpretation, here.

The goal is to predict if the student got a passing grade.

In [ ]:
df_student = pd.read_csv("../data/student.csv")
df_student.head()

> NB: there are no missing data

In [ ]:
X = df_student.drop(columns = ['passing'])
y = df_student['passing']

In [ ]:
## One Hot Encoding of all the categorical features
XOH = pd.get_dummies(X)


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(XOH,y,stratify=y, random_state=42)

y_test.value_counts()

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# If I don't increase the number of iterations I get a warning :-(
lr = LogisticRegression(max_iter=10**3)

knn = KNeighborsClassifier()


lr.fit(X_train,y_train)
knn.fit(X_train,y_train)


print( "Logistic regression accuracy on test set:" , accuracy_score( y_test , lr.predict( X_test ) ) )
print( "KNN accuracy on test set:" , accuracy_score( y_test , knn.predict( X_test ) ) )

print("the logistic regression gives better performance, so we use this as our final model.")

<br>

---

**Solution:**

List of the problems:

In [ ]:
# %load -r 1-20 solutions/solution_03_student.py


Proposition of code solving the problems:

In [ ]:
# %load -r 21-43 solutions/solution_03_student.py

In [ ]:
# %load -r 44-69 solutions/solution_03_student.py

In [ ]:
# %load -r 70-88 solutions/solution_03_student.py

In [ ]:
# %load -r 89- solutions/solution_03_student.py

[Back to ToC](#toc)

# Exercise: predicting 10 year coronary heart disease outcome <a class="anchor" id="exercise-heart"></a>

The [framingham dataset](https://datacatalog.med.nyu.edu/dataset/10046) links some patient features to their risk to develop a heart disease.

In [ ]:
df_heart=pd.read_csv('../data/framingham.csv')
print("{} samples - {} features".format(*(df_heart.shape)))
df_heart.head()

In [ ]:
##separation in X and y
X_heart = df_heart.drop( columns = "TenYearCHD" )
y_heart = df_heart[ "TenYearCHD" ]



Build and optimize a classifier to predict the column `'TenYearCHD'` (dependent variable : ten year risk of coronary heart disease).
 * use a KNN classifier or Logistic regression, ideally, compare them and choose the best.
 * this dataset has some missing values, decide how you want to handle them.
 * which metric do you want to use here?

***


**Hint :** before, your grid_param was a dictionnary, and it was specific to a particular model since parameters are model dependant. 
To adapt to multiple models, make a list of `grid_params` where each instance of the list is a dictionnary of parameters specific to the model, you want to try.
*Example:*
``` python
grid_param = [
                {"classifier": [KNeighborsClassifier()],
                 "classifier__n_neighbors": np.arange(1,30,1),
                 },
                {"classifier": [LogisticRegression()],
                 "classifier__penalty": ['l2','l1'],
                 }]
```

***
This is not an easy problem. **Do not despair if you do not get great performance out of your model, but focus on respecting best practices** (avoid leakage, account for imbalance,...)

***

Solution:

import and minimalist EDA

In [ ]:
# %load -r 1-19 solutions/solution_03_heart.py

splitting train and test

In [ ]:
# %load -r 20-29 solutions/solution_03_heart.py
## splitting train and test
X_heart_train, X_heart_test, y_heart_train, y_heart_test = train_test_split(X_heart,y_heart,
                                                                            stratify=y_heart)
#stratify is here to make sure that you split keeping the repartition of labels unaffected

print(f"full : {sum(y_heart)} CHD / {len(y_heart)} samples")
print(f"train: {sum(y_heart_train)} CHD / {len(y_heart_train)} samples")
print(f"test : {sum(y_heart_test)} CHD / {len(y_heart_test)} samples")
print("\n***\n")


creating the pipeline and grid, and fitting it

In [ ]:
# %load -r 30-55 solutions/solution_03_heart.py

looking at the best model

In [ ]:
# %load -r 58-87 solutions/solution_03_heart.py

getting a baseline for understanding the model performance

In [ ]:
# %load -r 88-100 solutions/solution_03_heart.py

checking feature importance to learn more about the problem

In [ ]:
# %load -r 101-115 solutions/solution_03_heart.py

[Back to ToC](#toc)


# Appendices <a class="anchor" id="APPENDIX"></a>

# Support Vector Machine (SVM) <a class="anchor" id="SVM"></a>

"The basic principle of SVM is pretty simple. SVM aims at finding the 'good' threshold (hyperplane) to separate data from different classes. Conceptually it is very different from logistic regression where you maximize the log likelihood of the log odds function. **With SVM you really look for an hyperplane that separates data and that's it : there is no underlying hypothesis about probability distribution or anything else. It is very geometrical.**

So what's a good threshold? Again it is going to depend on the metric you are interested in. But at least a good threshold should be linked to this biais variance trade off in the sens that it should offer flexibility to your model.

You can imagine that there is a quite a lot of hyperplanes separating data in your training set. You could stick your threshold right where the class 0 point closest to class 1 lies. But in that case it will be very far from the other class 0 points, which can be a problem. **You could decide that your threshold should be right between the two closest extreme of your classes but that is going to be very sensitive to missclassified data or extreme events... Those points choosen as a reference to put your threshold are called support vectors.**

In [ ]:
np.random.seed(10815657)
fig,ax = plt.subplots(1,2,figsize=(15,7))

# case 1 
norm1=0.2*np.random.randn(100)-2
norm2=0.8*np.random.randn(100)+2.5

# case 2
cauch=0.8*np.random.standard_cauchy(10)-2
norm=1*np.random.randn(100)+2.5

for i,Ds in enumerate( [ (norm1,norm2) , (cauch,norm) ] ):

    ax[i].plot(Ds[0],[1]*len(Ds[0]),'ro',markersize=10)
    ax[i].plot(Ds[1],[1]*len(Ds[1]),'bo')

    min2 = min( Ds[1] )
    max1 = max( Ds[0] )
    
    ax[i].axvline( min2 , color='k', linestyle='--', label='defined by the most extreme blue point')
    ax[i].axvline( (min2 +max1)/2,color='k',linestyle='-.',label='middle of extreme of two classes')
    ax[i].legend(loc='best')


So, once again, you are confronted to a compromise. You should place your threshold somwhere that is globally best even though that would mean some miss-classification. We are back to our regularization problem and of course **Support vector machine has a regularization parameter : C**. The game now becomes placing your threshold right in the middle of points (support vectors) from  each classes that you have \"chosen\" to be general points for decision making : **they don't need to be the two closest points from different classes anymore. They need to be points where your hyperplane makes the least error differentiating classes.**


![svm_margin](../images/SVM_margin.png)

Image source : image by wikipedia user Larhmam, distributed under a [CC BY-SA 4.0 license](https://creativecommons.org/licenses/by-sa/4.0/deed.en).


So you want to maximize the margin separating the two classes. This margin is $\frac{2}{||\pmb{w}||}$. So you want to minimize $||\pmb{w}||$. The SVM loss function we want to minimize with respect to $\pmb{w}$ and $b$ is:

$C\cdot\Sigma^{N}_{i=1}\zeta_i + \frac{1}{2}||\pmb{w}||^{2}$ subject to $\zeta_i \ge 0$ and $y_{i}(w^{T}x_{i}-b) \ge 1-\zeta_i$, where $\zeta_i = \Sigma^{N}_{i=1}max(0,1-y_{i}(\pmb{w}\cdot\pmb{x}_i-b))$
 * $y_i$ is $-1$ or $1$ depending on the class of the point $i$
 * the class of point $\pmb{x}$ is determined by the SVM using the sign of $(\pmb{w}\cdot\pmb{x}-b)$ (ie, on which side of the $(\pmb{w}\cdot\pmb{x}-b)$ hyperplane we are).



Note that you could also use a L1 regularization but it is not implemented in the function we are going to use.

Indeed if most of the data points are well separated in term of class on each side of the hyperplane then
* most of the time $y_{k}(w^{T}x_{k}-b) \geq 1$ and so $max(0,1-y_{k}(w^{T}x_{k}-b)=0$ (that's good for minimizing our loss function), 
* and a few times $y_{k}(w^{T}x_{k}-b) \leq -1$ and so $max(0,1-y_{k}(w^{T}x_{k}-b) \geq 2$ (which is polluting our minimization of the loss function).



You can see that there is a [dot product](https://en.wikipedia.org/wiki/Dot_product) involved : in the case of a linear hyperplane this dot product is just the cartesian dot product that you probably use all the time. It allows you to calculate distances between points in that cartesian space or between points and hyperplanes. But you might be familiar with other scalar product : like for example when you proceed to a Fourier decomposition of a function. This particular scalar product acts on functions and so is not really of interest for us... But others exist.

**So in principle you could use other definitions of distance between points to answer that classification question**. This is what non-linear SVM does and this is why you can choose different so called kernels as hyperparameters as we will see below :

$\overrightarrow{x_{i}}.\overrightarrow{x_{j}}$ : cartesian

$(\overrightarrow{x_{i}}.\overrightarrow{x_{j}})^{d}$ : polynomial degree d

$exp(-\gamma||\overrightarrow{x_{i}}-\overrightarrow{x_{j}}||^{2})$ : gaussian radial basis

$tanh(\kappa\overrightarrow{x_{i}}.\overrightarrow{x_{j}}+c)$ : hyperbolic tangent

**This is really powerful for classification but going non-linear by using a kernel trick prevents you from interpreting how your features are massaged to create this classifier... So, if you want interpretability and do science rather than engineering : keep it linear.**


![3d_svm](../images/3d_svm.png)



### Toy data : linear kernel <a class="anchor" id="SVM-linear"></a>

In [ ]:
from utils import contour_SVM
from sklearn.datasets import make_blobs
X_toy_2, y_toy_2 = make_blobs(n_samples=120, centers=2,cluster_std=3, random_state=6)

#2 classes
#parameters:X,y,C,kernel,degree for polynomial kernel,gamma for radial kernel,multi class strategy
contour_SVM(X_toy_2,y_toy_2,c=1,ker='linear')

In [ ]:
#2 classes
contour_SVM(X_toy_2,y_toy_2,c=0.01,ker='linear')

Reducing C to 0.01 reduces the cost of misclassification and makes the margin larger.

In [ ]:
#let's put very little regularization
contour_SVM(X_toy_2,y_toy_2,c=1000,ker='linear')

The inverse is true. Increasing C does not make a big difference to C=1.

### Toy data non linear kernel <a class="anchor" id="SVM-nonlinear"></a>

In [ ]:
#let's change the scalar product and the notion of distance by using a rbf kernel
#Also we did'nt put much of regularization
#parameters:X,y,C,kernel,degree for polynomial kernel,gamma for radial kernel,multi class strategy
contour_SVM(X_3,y_3,c=1,ker='rbf',gam=1)

In the above example we changed the kernel from linear to Gaussian radial basis. See how the classification boundaries follow local class density levels?

In [ ]:
#even less regularization
contour_SVM(X_3,y_3,c=1,ker='rbf',gam=10)

In the above example we greatly reduce the standard deviation on the gaussian model (we increased $\gamma$). See how now the boundaries are really concentrated around each point? 

In [ ]:
#more regularization
contour_SVM(X_3,y_3,c=1,ker='rbf',gam=0.1)

Inversely by expanding the variance in the gaussian we end up with larger chunks.

In [ ]:
#little regularization but a polynomial kernel of degree 3 this time
contour_SVM(X_3,y_3,c=1,ker='poly',deg=3)

Finally, above we tried a degree 3 polynomial kernel?

### Cancer dataset <a class="anchor" id="SVM-hyperparameters"></a>

In [ ]:
scaler = StandardScaler()
scaler.fit(X_cancer_train)
X_cancer_train_scaled = scaler.transform(X_cancer_train)
X_cancer_test_scaled = scaler.transform(X_cancer_test)



In [ ]:
from sklearn.metrics import roc_auc_score
C_range=np.logspace(-2,3,50)


scores_test=[]
scores_train=[]

for k in C_range: 
    SVM_r = svm.SVC(C=k, kernel='linear', 
                    class_weight='balanced', probability=True)
    SVM_r.fit(X_cancer_train_scaled,y_cancer_train)

    y_train_prob = SVM_r.predict_proba(X_cancer_train_scaled)
    y_test_prob = SVM_r.predict_proba(X_cancer_test_scaled)

    train_roc_auc_score=roc_auc_score(y_cancer_train,y_train_prob[:, 1])
    test_roc_auc_score=roc_auc_score(y_cancer_test,y_test_prob[:, 1])

    scores_test.append(test_roc_auc_score)
    scores_train.append(train_roc_auc_score)

plt.figure(figsize=(10,10))
plt.xlabel('inverse of the l2 weight')
plt.ylabel('ROC AUC')
plt.plot(C_range, scores_train,'k-',linewidth=5,label='train')
plt.plot(C_range, scores_test,'r-',linewidth=5,label='test')
plt.xscale('log')
plt.legend(loc='best')
plt.show()

In [ ]:
gamma_range=np.logspace(-3,3,50)

scores_test=[]
scores_train=[]

for k in gamma_range: 
    SVM_r = svm.SVC(C=k, kernel='rbf', gamma=k ,
                    class_weight='balanced', probability=True)
    SVM_r.fit(X_cancer_train_scaled,y_cancer_train)

    y_train_prob = SVM_r.predict_proba(X_cancer_train_scaled)
    y_test_prob = SVM_r.predict_proba(X_cancer_test_scaled)

    train_roc_auc_score=roc_auc_score(y_cancer_train,y_train_prob[:, 1])
    test_roc_auc_score=roc_auc_score(y_cancer_test,y_test_prob[:, 1])

    scores_test.append(test_roc_auc_score)
    scores_train.append(train_roc_auc_score)
    
plt.figure()
plt.xlabel('$\gamma$ in rbf model')
plt.ylabel('ROIC AUC')
plt.plot(gamma_range, scores_train,'k-',linewidth=5,label='train')
plt.plot(gamma_range, scores_test,'r-',linewidth=5,label='test')
#plt.xticks([0,20,40,60,80,100])
plt.xscale('log')
plt.legend(loc='best')
plt.show()

In [ ]:
degree_range=np.arange(0,10,1)

scores_test=[]
scores_train=[]

for k in degree_range: 
    SVM_r = svm.SVC(C=1, kernel='poly', degree=k, 
                    probability=True, class_weight='balanced')
    SVM_r.fit(X_cancer_train_scaled,y_cancer_train)
    y_train_prob = SVM_r.predict_proba(X_cancer_train_scaled)
    y_test_prob = SVM_r.predict_proba(X_cancer_test_scaled)

    train_roc_auc_score=roc_auc_score(y_cancer_train,y_train_prob[:, 1])
    test_roc_auc_score=roc_auc_score(y_cancer_test,y_test_prob[:, 1])

    scores_test.append(test_roc_auc_score)
    scores_train.append(train_roc_auc_score)

plt.figure()
plt.xlabel('degree in poly model')
plt.ylabel('ROC AUC')
plt.plot(degree_range, scores_train,'k-',linewidth=5,label='train')
plt.plot(degree_range, scores_test,'r-',linewidth=5,label='test')
#plt.xticks([0,20,40,60,80,100])
#plt.xscale('log')
plt.legend(loc='best')
plt.show()

Those 3 hyperparameters show the usual overfitting behaviour when they are increased : monotonous increase of the accuracy on the train function whereas there is a maximum accuracy for the test function after which accuracy decreases with increasing hyperpaprameter value.

If you want to get the weights from the SVM just use `coef_` again but it will only work if you are in the context of a linear SVC.

# String kernels <a class="anchor" id="SVM-kernels"></a>

This paragraph illustrates how to use a SVM kernel not present in sklearn.svm. The following code uses the string kernel from [Hilmarsson et al. BioRxiv, 2021](https://doi.org/10.1101/2021.09.19.460980) in order to classify genomic sequences (here we use toy data based on patterns from real human samples). 

![Figure 2 of Hilmarsson et al. BioRxiv, 2021 : illustration of the string kernel computation](../images/stringKernel.png)

The code is copied from https://github.com/weekend37/string-kernels/blob/master/example.ipynb. More interesting examples can be found in the paper. First let's install the stringkernel package:

In [ ]:
#to install string-kernel if that is not already done
!python3 -m pip install string-kernels --upgrade

Let's load the data:

In [ ]:
import os 

data_path = "../data"

samples_train = np.load(os.path.join(data_path, "samples_train"+".npy"))
samples_validation = np.load(os.path.join(data_path, "samples_validation"+".npy"))

ancestry_train = np.load(os.path.join(data_path, "ancestry_train"+".npy"))
ancestry_validation = np.load(os.path.join(data_path, "ancestry_validation"+".npy"))

reference = np.load(os.path.join(data_path, "reference"+".npy"))
populations = np.load(os.path.join(data_path, "populations"+".npy"))

print("number of training samples:", len(samples_train))
print("number of validation samples:", len(samples_validation))
print("Sequence length:", len(reference))
print("Number of ancestries:", len(populations))

The data matrix consists of arrays of nucleotides (rows), where each columns corresponds to a sequence position and each row to a sample.

In [ ]:
samples_train

To use the standart SVM with RBF we need to turn the sequences into numeric values. For this example, we just store the information whether the sequences are the same (0) or different (1) from the reference sequence.

In [ ]:
X_train = (samples_train != reference).astype(int)
X_train

The labels are balanced:

In [ ]:
from stringkernels.utils import plot_label_distribution

plot_label_distribution(ancestry_train)
np.unique(ancestry_train)

The labels need to be turned into numbers in order to use the SVC class with RBF kernel:

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(ancestry_train)
y_train = le.transform(ancestry_train)
np.unique(y_train)

In [ ]:
X_val = (samples_validation != reference).astype(int)
y_val = le.transform(ancestry_validation)

In [ ]:
from sklearn import svm
from sklearn.metrics import accuracy_score

svm_model = svm.SVC(kernel='rbf')
svm_model.fit(X_train, y_train)
y_pred = svm_model.predict(X_val)
svm_accuracy = accuracy_score(y_val, y_pred)
print("Validation Accuracy: {}%".format(np.round(svm_accuracy*100, 2)))


To use SVC with the string kernel we encountered in the lecture, we do not need the make the features numeric, but we can pass the string as character arrays directly:

In [ ]:
from stringkernels.kernels import string_kernel

svm_sk_model = svm.SVC(kernel=string_kernel())
svm_sk_model.fit(samples_train, y_train)
y_pred = svm_sk_model.predict(samples_validation)
svm_sk_accuracy = accuracy_score(y_val, y_pred)
print("Validation Accuracy: {}%".format(np.round(svm_sk_accuracy*100, 2)))

In [ ]:
from stringkernels.utils import plot_accuracies

plot_accuracies({
    "SVM\n RBF-Kernel": svm_accuracy,
    "SVM\n String-Kernel": svm_sk_accuracy,
})

[Back to the ToC](#toc)

## Annex : Multiclass dataset : penguins identification <a id='data-penguin'></a>

This dataset consists in data from 344 penguins grouped in three species :

![penguins](../images/lter_penguins.png)
*Artwork by @allison_horst*

[dataset source](https://allisonhorst.github.io/palmerpenguins/)


Among the columns present in there, we drop the ones, like `StudyID` or `sampleNumber` to keep only columns of potential interest for the task at hand :

In [ ]:
df_penguin = pd.read_csv( "../data/horst2020_palmerpenguins_raw.csv" )

kept_columns = [ 'Species',  'Clutch Completion', 'Culmen Length (mm)',
       'Culmen Depth (mm)', 'Flipper Length (mm)', 'Body Mass (g)', 'Sex',
       'Delta 15 N (o/oo)', 'Delta 13 C (o/oo)']


## keep the nice doodle color code!
penguin_colors = {'Adelie':'#FF8C00',
                  'Chinstrap':"#A020F0",
                  'Gentoo':"#008B8B"}


df_penguin = df_penguin.loc[: , kept_columns ]
# keep only the first word of the species field. We do not need the latin name here
df_penguin.Species = df_penguin.Species.apply(lambda x: x.partition(' ')[0])
df_penguin.head()

The columns correspond to :

* **Species** : the penguin species
* **Clutch Completion** : if the study nest ws observed with a full clutch, i.e., 2 eggs
* **Culmen Length** : length of the dorsal ridge of a bird's bill (millimeters)
* **Culmen Depth** : the depth of the dorsal ridge of a bird's bill (millimeters)
* **Flipper Length** : the length of a bird's flipper (millimeters)
* **Body Mass** : the penguin body mass (grams)
* **Sex** : the sex of the animal
* **Delta 15 N** : measure of the ratio of stable isotopes 15N:14N
* **Delta 13 C** : measure of the ratio of stable isotopes 13C:12C

You can see there are some NAs. Let's throw away samples with Nas (only 20 of them):

In [ ]:
K = df_penguin.isnull().sum(axis=1) < 1

df_penguin = df_penguin.loc[K,:]
df_penguin.shape

In [ ]:
df_penguin.Species.value_counts()

In [ ]:
## finally, we separate the target variable (Species) from the rest
y_penguin = df_penguin.Species

# here note the usage of pd.get_dummies to one-hot encode categorical data!
X_penguin = pd.get_dummies(df_penguin.drop( columns="Species" ), drop_first=True)

In [ ]:
X_penguin

We can also look at the structure of the data by plotting the correlation between the features:

In [ ]:
df_penguin_corr = df_penguin.corr(numeric_only=True)

sns.clustermap(df_penguin_corr,
               figsize=(8,8),
               z_score=None,
               row_cluster=True,
               col_cluster=True,
               method='ward',
               cmap='coolwarm',vmax=1,vmin=-1, 
               annot=True, annot_kws={"size": 13},cbar_kws={"label": 'Pearson\ncorrelation'})
## sns allows you to do a hierarchical clustering that simply
plt.show()

The above pearson correlation matrix is highly informational. 

It allows you to see that **some variables are probably redundant** and so you don't need to have all of them, hence reducing the complexity of your model. 

Moreover it is usually bad to have highly correlated variables in your model since it is making it unstable and less interpretable. 

**micro-exercise :** what could we use to handle/mitigate this problem ?

[Back to the ToC](#toc)

# Exercise <a class="anchor" id="exo"></a>


Using either the **cancer or the penguin dataset** compare and find the 'best' classifier. What do you learn from those classifiers? 

Don't forget that you have also seen stuff about preprocessing.

Don't go too crazy as it can be time consuming: limit the number of parameter values you are testing for.

**Hint :** before, your grid_param was a dictionnary, and it was specific to a particular model since parameters are model dependant. 
To adapt to multiple models, make a list of `grid_params` where each instance of the list is a dictionnary of parameters specific to the model, you want to try.
*Example:*
``` python
grid_param = [
                {"classifier": [KNeighborsClassifier()],
                 "classifier__n_neighbors": np.arange(1,30,1),
                 },
                {"classifier": [LogisticRegression()],
                 "classifier__penalty": ['l2','l1'],
                 }]
```

**Additionnal question** : How would you work with your PCA data and incorporate it to your model?

*Hint:* 
 * Don't forget that PCA needs a centering and a scaling step...
 * Adding PCA can be quite time consuming too. Try a small number of pca components first with other hyperparameters centered around what you found before!


In [ ]:
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV


## PENGUIN CORRECTION

setup the pipeline and fit it :

In [ ]:
# %load -r -24 solutions/solution_02_penguin.py

reporting best model:

In [ ]:
# %load -r 25- solutions/solution_02_penguin.py

## CANCER CORRECTION 

build and fit the pipeline:

In [ ]:
# %load -r -26 solutions/solution_02_cancer.py

report the best model:

In [ ]:
# %load -r 28-45 solutions/solution_02_cancer.py

plot a roc curve:

In [ ]:
# %load -r 46-47 solutions/solution_02_cancer.py

**Alternative with PCA:**

In [ ]:
# %load -r 51-89 solutions/solution_02_cancer.py

In [ ]:
# %load -r 90-104 solutions/solution_02_cancer.py

In [ ]:
# %load -r 106- solutions/solution_02_cancer.py